In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [4]:
df = fetch_hourly_rides(12)

2025-03-04 21:44:50,680 INFO: Initializing external client
2025-03-04 21:44:50,680 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:44:53,363 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215644
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.91s) 


In [5]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-04 16:00:00+00:00,83,0
1,2025-03-04 20:00:00+00:00,262,30
2,2025-03-04 18:00:00+00:00,68,218
3,2025-03-04 14:00:00+00:00,186,164
4,2025-03-04 12:00:00+00:00,131,0
...,...,...,...
4262,2025-03-05 01:00:00+00:00,256,0
4263,2025-03-05 01:00:00+00:00,31,0
4264,2025-03-05 01:00:00+00:00,28,0
4265,2025-03-05 01:00:00+00:00,224,1


In [6]:
df_pred = fetch_predictions(12)

2025-03-04 21:44:59,778 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 21:44:59,812 INFO: Initializing external client
2025-03-04 21:44:59,821 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:45:01,730 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215644
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.24s) 


In [7]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,199,0.0,2025-03-04 09:00:00+00:00
1,30,0.0,2025-03-04 09:00:00+00:00
2,32,0.0,2025-03-04 09:00:00+00:00
3,60,0.0,2025-03-04 09:00:00+00:00
4,31,0.0,2025-03-04 09:00:00+00:00
...,...,...,...
999,122,0.0,2025-03-05 02:00:00+00:00
1000,55,0.0,2025-03-05 02:00:00+00:00
1001,263,132.0,2025-03-05 02:00:00+00:00
1002,89,0.0,2025-03-05 02:00:00+00:00


In [8]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [9]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-04 09:00:00+00:00,190,1,0.0
1,2025-03-05 00:00:00+00:00,38,0,0.0
2,2025-03-04 09:00:00+00:00,178,0,0.0
3,2025-03-05 00:00:00+00:00,47,0,0.0
4,2025-03-04 09:00:00+00:00,181,1,3.0
...,...,...,...,...
748,2025-03-05 01:00:00+00:00,256,0,1.0
749,2025-03-05 01:00:00+00:00,31,0,0.0
750,2025-03-05 01:00:00+00:00,28,0,1.0
751,2025-03-05 01:00:00+00:00,224,1,8.0


In [10]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [11]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
383,2025-03-04 09:00:00+00:00,2,0,0.0,0.0
409,2025-03-05 00:00:00+00:00,2,0,0.0,0.0
721,2025-03-05 01:00:00+00:00,2,0,0.0,0.0
96,2025-03-04 09:00:00+00:00,3,0,0.0,0.0
458,2025-03-05 00:00:00+00:00,3,0,0.0,0.0
...,...,...,...,...,...
356,2025-03-05 00:00:00+00:00,262,1,32.0,31.0
614,2025-03-05 01:00:00+00:00,262,2,64.0,62.0
83,2025-03-04 09:00:00+00:00,263,134,101.0,-33.0
257,2025-03-05 00:00:00+00:00,263,22,24.0,2.0


In [12]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
0,2025-03-04 09:00:00+00:00,190,1,0.0,-1.0
1,2025-03-05 00:00:00+00:00,38,0,0.0,0.0
2,2025-03-04 09:00:00+00:00,178,0,0.0,0.0
3,2025-03-05 00:00:00+00:00,47,0,0.0,0.0
4,2025-03-04 09:00:00+00:00,181,1,3.0,2.0
...,...,...,...,...,...
748,2025-03-05 01:00:00+00:00,256,0,1.0,1.0
749,2025-03-05 01:00:00+00:00,31,0,0.0,0.0
750,2025-03-05 01:00:00+00:00,28,0,1.0,1.0
751,2025-03-05 01:00:00+00:00,224,1,8.0,7.0


In [13]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [14]:
mae_by_hour["MAE"].mean()

13.508632138114208